In [4]:
# importing pandas, numpy, matplotlib, seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

In [5]:
data=pd.read_csv("hospital_patients_real_world.csv")

In [6]:
data.head()

,PatientID,Age,Gender,Diagnosis,AdmissionDate,DischargeDate,HospitalID
0,PN-2021066,7.0,Other,Myocardial Infarction,2024-03-23,2024-03-29,HOSP-65
1,PN-4606019,36.0,Other,Pneumonia,2024-08-01,2024-08-07,HOSP-79
2,PN-2594016,70.0,Other,Influenza,2024-11-16,2024-11-23,HOSP-27
3,PN-6906914,90.0,Unknown,Acute Bronchitis,2025-07-05,2025-07-10,HOSP-64
4,PN-4656204,0.0,Female,Type 2 Diabetes,2023-08-30,2023-08-31,HOSP-31


In [7]:
data.columns

Index(['PatientID', 'Age', 'Gender', 'Diagnosis', 'AdmissionDate',
       'DischargeDate', 'HospitalID'],
      dtype='str')

In [8]:
print(data.size)
data.shape

35000


(5000, 7)

In [9]:
data.describe()

,Age
count,4650.000000
mean,47.384301
std,27.880535
min,0.000000
25%,23.000000
50%,47.000000
75%,72.000000
max,95.000000


In [10]:
data['Age'].isnull().sum()

np.int64(350)

In [11]:
# Look at records with age = 0
zero_age_records = data[data['Age'] == 0]
print(f"Number of age = 0: {len(zero_age_records)}")

# Check their diagnoses
print(zero_age_records[['Age', 'Diagnosis', 'Gender']].head(20))

Number of age = 0: 52
      Age                Diagnosis   Gender
4     0.0          Type 2 Diabetes   Female
222   0.0         Acute Bronchitis  Unknown
547   0.0                   Asthma   Female
640   0.0         Acute Bronchitis   Female
689   0.0             Hypertension    Other
725   0.0    Myocardial Infarction    Other
815   0.0  Urinary Tract Infection  Unknown
865   0.0           Osteoarthritis   Female
936   0.0           Osteoarthritis    Other
975   0.0   Chronic Kidney Disease  Unknown
977   0.0                Pneumonia  Unknown
1016  0.0             Hypertension   Female
1037  0.0  Urinary Tract Infection    Other
1104  0.0                Influenza  Unknown
1169  0.0           Diverticulitis   Female
1449  0.0         Acute Bronchitis    Other
1452  0.0  Urinary Tract Infection   Female
1476  0.0      Atrial Fibrillation     Male
1484  0.0                      NaN   Female
1775  0.0          Type 2 Diabetes     Male


these are not sicknesses for newborns.so 0 means none surely


In [12]:
# What is the 75th percentile age?
percentile_75 = data['Age'].quantile(0.75)
print(f"75th percentile age: {percentile_75}")


75th percentile age: 72.0


72 age seems to be an old age.so it seems harmful to change all zeros to this.we stick to mean


In [13]:
print(f"Age = 0 count before: {(data['Age'] == 0).sum()}")

median_age = data[data['Age'] > 0]['Age'].median()
print(f"Median age (excluding zeros): {median_age}")

# Replace all zeros with this median
data['Age'] = data['Age'].replace(0, median_age)

# Verify
print(f"Zeros remaining: {(data['Age'] == 0).sum()}")
print(f"Age = 0 count after : {(data['Age'] == 0).sum()}")

Age = 0 count before: 52
Median age (excluding zeros): 48.0
Zeros remaining: 0
Age = 0 count after : 0


In [14]:
data['Age'] = data['Age'].replace(0, np.nan)

# Fill with mean of each gender group (more balanced)
data['Age'] = data.groupby('Gender')['Age'].transform(lambda x: x.fillna(x.mean()))

In [15]:
data['Age'].fillna(data['Age'].mean())

0        7.000000
1       36.000000
2       70.000000
3       90.000000
4       48.000000
          ...    
4995    22.000000
4996    48.070859
4997    48.000000
4998    29.000000
4999    93.000000
Name: Age, Length: 5000, dtype: float64

In [16]:
data.info()

<class 'pandas.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   PatientID      5000 non-null   str    
 1   Age            4650 non-null   float64
 2   Gender         4650 non-null   str    
 3   Diagnosis      4650 non-null   str    
 4   AdmissionDate  5000 non-null   str    
 5   DischargeDate  5000 non-null   str    
 6   HospitalID     5000 non-null   str    
dtypes: float64(1), str(6)
memory usage: 273.6 KB


In [17]:
data['AdmissionDate'] = pd.to_datetime(data['AdmissionDate'])
data['DischargeDate'] = pd.to_datetime(data['DischargeDate'])

In [18]:
# Find where discharge is BEFORE admission (illogical!)
invalid_dates = data[data['DischargeDate'] < data['AdmissionDate']]

print(f"Records with discharge before admission: {len(invalid_dates)}")
print(invalid_dates[['AdmissionDate', 'DischargeDate']].head(10))

Records with discharge before admission: 150
    AdmissionDate DischargeDate
23     2026-01-13    2026-01-08
26     2024-06-04    2024-05-30
41     2023-07-25    2023-07-20
47     2024-01-12    2024-01-07
105    2025-06-25    2025-06-20
117    2024-01-21    2024-01-16
185    2024-05-01    2024-04-26
201    2024-11-26    2024-11-21
202    2023-09-13    2023-09-08
212    2023-11-17    2023-11-12


In [19]:
# Swap admission and discharge for invalid records
mask = data['DischargeDate'] < data['AdmissionDate']
data.loc[mask, ['AdmissionDate', 'DischargeDate']] = data.loc[mask, ['DischargeDate', 'AdmissionDate']].values

In [20]:
# Check Gender variations
print(data['Gender'].value_counts())
print(data['Gender'].unique())
print(f"Unique values: {data['Gender'].nunique()}")

Gender
Other      1223
Unknown    1163
Female     1153
Male       1111
Name: count, dtype: int64
<StringArray>
['Other', 'Unknown', 'Female', 'Male', nan]
Length: 5, dtype: str
Unique values: 4


In [21]:
# Check for variations with spaces or capitalization
print(data['Gender'].unique())
print(f"Unique values: {data['Gender'].nunique()}")

# Check for leading/trailing spaces
print(data['Gender'].str.strip().value_counts())

<StringArray>
['Other', 'Unknown', 'Female', 'Male', nan]
Length: 5, dtype: str
Unique values: 4
Gender
Other      1223
Unknown    1163
Female     1153
Male       1111
Name: count, dtype: int64


In [22]:
data['Gender'] = data['Gender'].replace({'Other':'Unknown' })
data.head()

,PatientID,Age,Gender,Diagnosis,AdmissionDate,DischargeDate,HospitalID
0,PN-2021066,7.0,Unknown,Myocardial Infarction,2024-03-23,2024-03-29,HOSP-65
1,PN-4606019,36.0,Unknown,Pneumonia,2024-08-01,2024-08-07,HOSP-79
2,PN-2594016,70.0,Unknown,Influenza,2024-11-16,2024-11-23,HOSP-27
3,PN-6906914,90.0,Unknown,Acute Bronchitis,2025-07-05,2025-07-10,HOSP-64
4,PN-4656204,48.0,Female,Type 2 Diabetes,2023-08-30,2023-08-31,HOSP-31
